In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import OPTICS


In [ ]:
# load in csv
df = pd.read_csv('6-class-csv.csv')
features = df.iloc[:, [i for i in range(df.shape[1]) if i != 4]]
labels = df.iloc[:, 4]

print(labels)
print(features)

# Hertzsprung-Russell Diagram

In [ ]:
sns.set(style="whitegrid", context="notebook")
plt.figure(figsize=(10, 7))

scatter = plt.scatter(df["Temperature (K)"], df["Luminosity(L/Lo)"], c=df["Star type"], cmap="plasma", s=40, edgecolors="k", alpha=0.8)

# weird axes settings for HR diagram
plt.gca().invert_xaxis() # reverse because temp decreases
plt.yscale("log") # log scale for luminosity

# titles/labels
plt.title("Hertzsprung–Russell Diagram", fontsize=16)
plt.xlabel("Temperature (K)", fontsize=14)
plt.ylabel("Luminosity (L/Lo)", fontsize=14)

# add star type color bar
cbar = plt.colorbar(scatter)
cbar.set_label("Star Type")

# add annotations for star types
plt.text(3000, 1e5, "Supergiants", color="black", fontsize=10)
plt.text(6000, 1, "Main Sequence", color="black", fontsize=10)
plt.text(10000, 1e-3, "White Dwarfs", color="black", fontsize=10)

plt.tight_layout()
plt.show()

# Basic Statistics

In [ ]:
# Basic Statistics
print("Basic Statistics")
print(df.describe())
print("Class Distribution")
print(df['Star type'].value_counts())
print("Missing Values")
print(df.isnull().sum())

In [ ]:
# Dictionary mapping for star colors to integers
star_color_mapping = {color: idx for idx, color in enumerate(features['Star color'].unique())}
features_num=features.copy()
features_num['Star color'] = features_num['Star color'].map(star_color_mapping)

spectral_class_mapping = {s_class: idx for idx, s_class in enumerate(features['Spectral Class'].unique())}
features_num['Spectral Class'] = features_num['Spectral Class'].map(spectral_class_mapping)

In [ ]:
# Bar charts
star_type_counts = df['Star type'].value_counts().sort_index()
plt.figure(figsize=(8, 5))
sns.barplot(x=star_type_counts.index, y=star_type_counts.values, palette="viridis")
plt.title("Star Type Distribution", fontsize=16)
plt.xlabel("Star Type", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.tight_layout()
plt.show()
# OPTICS Clustering
print(features)
# Convert strings in features to numeric if necessary
optics_model = OPTICS(min_samples=10, xi=0.05, min_cluster_size=0.05)
optics_model.fit(features_num)
labels = optics_model.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Number of clusters found: {n_clusters}")
# Plot OPTICS clustering results
plt.figure(figsize=(10, 7))
unique_labels = set(labels)
colors = plt.cm.get_cmap('tab10', len(unique_labels))
for k in unique_labels:
    class_member_mask = (labels == k)
    xy = features[class_member_mask]
    plt.scatter(xy.iloc[:, 0], xy.iloc[:, 1], s=40, color=colors(k), label=f'Cluster {k}' if k != -1 else 'Noise', edgecolors='k', alpha=0.7)